# Статистический анализ данных, проектная работа. 
# Определение перспективного тарифа для телеком-компании

## Описание проекта
В нашем распоряжении данные 500 пользователей «Мегалайна»(федерального оператора сотовой связи): кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра».
Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.
Нам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

Описание тарифов

Тариф «Смарт»:

Ежемесячная плата: 550 рублей.
Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика.
Стоимость услуг сверх тарифного пакета:
Минута разговора: 3 рубля.
Сообщение: 3 рубля.
1 Гб интернет-трафика: 200 рублей.

Тариф «Ультра»:

Ежемесячная плата: 1950 рублей.
Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика.
Стоимость услуг сверх тарифного пакета:
Минута разговора: 1 рубль.
Сообщение: 1 рубль.
1 Гб интернет-трафика: 150 рублей.

«Мегалайн» всегда округляет секунды до минут, а мегабайты — до гигабайт. Каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута.
Для веб-трафика отдельные сессии не считаются. Вместо этого общая сумма за месяц округляется в бо́льшую сторону. Если абонент использует 1025 мегабайт в этом месяце, с него возьмут плату за 2 гигабайта.

## Шаг 1 . Откроем файлы с данными и изучим общую информацию.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from scipy import stats as st
import math

users = pd.read_csv('/datasets/users.csv')
calls = pd.read_csv('/datasets/calls.csv')
messages = pd.read_csv('/datasets/messages.csv')
internet = pd.read_csv('/datasets/internet.csv', index_col=[0])
tariffs = pd.read_csv('/datasets/tariffs.csv')

In [ ]:
users

In [ ]:
users.info()

В таблице представлена информация о пользователях, 8 колонок, 500 строк:
* user_id — уникальный идентификатор пользователя
* first_name — имя пользователя
* last_name — фамилия пользователя
* age — возраст пользователя (годы)
* reg_date — дата подключения тарифа (день, месяц, год)
* churn_date — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных)
* city — город проживания пользователя
* tariff — название тарифного плана

Имеются пропуски в столбце churn_date, они значат, что пользователи всё еще пользуются тарифом, нужно изменитть тип даннных.

In [ ]:
calls

In [ ]:
calls.info()

В таблице представлена информация о звонках, 202607 строки, 4 колонки:
* id — уникальный номер звонка
* call_date — дата звонка
* duration — длительность звонка в минутах
* user_id — идентификатор пользователя, сделавшего звонок

Пропусков нет, но нужно изменить тип данных.


In [ ]:
messages

In [ ]:
messages.info()

В таблице представлена информация о сообщениях, 123036 строк и 3 колонки:
* id — уникальный номер сообщения
* message_date — дата сообщения
* user_id — идентификатор пользователя, отправившего сообщение

Пропусков нет, но нужно изменить тип данных. 

In [ ]:
internet

In [ ]:
internet.info()

В таблице представлена информация об интернет-сессиях, 149396 строки, 4 колонки:
* id — уникальный номер сессии
* mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
* session_date — дата интернет-сессии
* user_id — идентификатор пользователя

Пропусков нет, но нужно изменить тип данных.




In [ ]:
tariffs

In [ ]:
tariffs.info()

В таблице представлена информация о тарифах, две строки, 8 колонок:
* tariff_name — название тарифа
* rub_monthly_fee — ежемесячная абонентская плата в рублях
* minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
* messages_included — количество сообщений в месяц, включённых в абонентскую плату
* mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
* rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
* rub_per_message — стоимость отправки сообщения сверх тарифного пакета
* rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)

Пропусков нет, но нужно изменить тип данных.




### Вывод: 
Отличный набор данных, нет потеряных данных и аномальных значений, нужно лишь проверить на дубликаты и заменить типы данных, где это необходимо.

## Шаг 2. Предобработка данных

Удалим явные дубликаты во всех таблицах, заменим типы данных и произведем округления согласно политике «Мегалайн»

In [ ]:
users.duplicated().sum()

In [ ]:
calls.duplicated().sum()

In [ ]:
messages.duplicated().sum()

In [ ]:
internet.duplicated().sum()

А дубликатов и нет

In [ ]:
users['tariff'].value_counts()

Неявных дубликатов в названиях тарифов тоже нет, пользователей с тарифом ultra более, чем вдвое меньше, чем пользователей с тарифом smart

Заменим типы данных и округлим длительность звонков и количество потребленного трафика в большую сторону

In [ ]:
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d')
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')

calls['id'] = calls['id'].astype(int)
calls['duration'] = calls['duration'].apply(np.ceil).astype(int)
calls['user_id'] = calls['user_id'].astype(int)
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')

messages['id'] = messages['id'].astype(int)
messages['user_id'] = messages['user_id'].astype(int)
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')

internet['id'] = internet['id'].astype(int)
internet['user_id'] = internet['user_id'].astype(int)
internet['mb_used'] = internet['mb_used'].apply(np.ceil).astype(int)
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d')


### Вывод:

Убедились, что нет дубликатов, заменили ошибочные типы данных на верные и округлили значения.

## Шаг 3. Подготовим данные, проведем необходимые рассчёты.

Изучим длительность звонков и количество интернет-траффика, посмотрим, как выглядят данные в диаграммах.

In [ ]:
calls.describe()

In [ ]:
calls['duration'].plot(kind='hist', bins = 38, figsize=(16, 8))

Видим, что много звонков длятся 0 минут, похоже, что это пропущенные вызовы. Так как минуты не расходуются, то и абонент за эти звонки  не платит, можно их смело удалить, для анализа они нам не понадобятся.

In [ ]:
calls = calls.query('duration != 0')

In [ ]:
internet.describe()

In [ ]:
internet['mb_used'].plot(kind='hist', bins = 50, figsize=(16, 8))

Здесь тоже видим много нулевых значений. Видимо, это факты установки соединения без расхода траффика, нет трафика - нет платы, эти данные также удалим.

In [ ]:
internet = internet.query('mb_used != 0')

Посчитаем количество израсходованных минут, смс, интернет трафика и выручки с каждого пользователя по месяцам

Добавим колонку с месяцами

In [ ]:
def month(df, col): 
    df = df.assign(month=df[col].dt.month)
    return df

Посчитаем количество звонков и минут, создадим таблицу по пользователям, количеству звонков и израсходованных минут по месяцам.

In [ ]:
calls = month(calls, 'call_date')

In [ ]:
calls_pivot = calls.pivot_table(index=['user_id', 'month'],
                                values='duration', aggfunc=("sum",'count')).reset_index()

calls_pivot.columns = ('user_id', 'month', 'calls_count', 'mins')
calls_pivot

Посчитаем количество отправленных смс, создадим таблицу по пользователям и количеству сообщений по месяцам

In [ ]:
messages = month(messages, 'message_date')

In [ ]:
messages_pivot = messages.pivot_table(index=['user_id','month'],
                                      values='message_date', aggfunc=('count')).reset_index()

messages_pivot.columns = ('user_id', 'month', 'messages_count')
messages_pivot

Посчитаем количество израсходованного трафика, создадим таблицу по пользователям и количеству израсходованных мегабайт по месяцам

In [ ]:
internet = month(internet, 'session_date')

In [ ]:
internet_pivot = internet.pivot_table(index=['user_id','month'],
                                      values='mb_used', aggfunc=('sum')).reset_index()

internet_pivot.head()

Посчитаем выручку.

In [ ]:
tariffs.columns = ['messages_included', 'mb_per_month_included', 'minutes_included',
       'rub_monthly_fee', 'rub_per_gb', 'rub_per_message', 'rub_per_minute',
       'tariff']

Объединим таблицы и посмотрим общую информацию.

In [ ]:
data_merge = calls_pivot.merge(messages_pivot, on=('user_id', 'month'), how='outer')
data_merge = data_merge.merge(internet_pivot, on=('user_id', 'month'), how='outer')
data_merge = data_merge.merge(users, on=('user_id'), how='left')
data_merge = data_merge.merge(tariffs, on=('tariff'), how='left')
data_merge.info()

Получили пропуски, они появились, потому что не все абоненты пользовались всеми услугами каждый месяц. Заполним пропуски нулями и заменим тип данных. 

In [ ]:
for col in ['calls_count', 'mins', 'messages_count','mb_used']:
    data_merge[col] = data_merge[col].fillna(0).astype(int)

Рассчитаем прибыль компании.

In [ ]:
def profit_calculation(df):
    df['profit'] = df['rub_monthly_fee']
    if df['mins'] > df['minutes_included']:
        df['profit'] += (df['mins'] - df['minutes_included']) * df['rub_per_minute']
    if df['messages_count'] > df['messages_included']:
        df['profit'] += (df['messages_count'] - df['messages_included']) * df['rub_per_message']
    if df['mb_used'] > df['mb_per_month_included']:
        df['profit'] += math.ceil((df['mb_used'] - df['mb_per_month_included']) / 1024) * (df['rub_per_gb'])    
    return df

In [ ]:
data_merge = data_merge.apply(profit_calculation, axis=1)
data_merge['profit'] = data_merge['profit'].astype(int)
data_merge

Удалим колонки, которые нам не нужны.

In [ ]:
data_merge = data_merge.drop(['first_name', 'last_name', 'messages_included', 'mb_per_month_included','minutes_included', 'rub_monthly_fee', 'rub_per_gb', 'rub_per_message', 'rub_per_minute'], axis=1)
data_merge.columns

### Вывод: 

Посчитали количество израсходованных минут, смс, интернет трафика и выручки с каждого пользователя по месяцам, выручку и прибыль компании. Очистили данные от ненужных значений.

## Шаг 4 Проведём анализ данных

Посчитаем среднее количество, дисперсию и стандартное отклонение для тарифов

In [ ]:
def func_pivot(value):
    table = data_merge.pivot_table(index=('tariff'), values=value,\
                       aggfunc=('median','mean','var','std')).astype(int).reset_index()
    return table

In [ ]:
mins_pivot = func_pivot('mins')
mins_pivot

In [ ]:
messages_pivot = func_pivot('messages_count')
messages_pivot

In [ ]:
internet_pivot = func_pivot('mb_used')
internet_pivot

Построим гистограммы

In [ ]:
data_merge[(data_merge['tariff'] == 'smart')
           & (data_merge['mins'] !=0)]['mins'].plot(kind='hist', bins=50, title='Минут в месяц', figsize=(16, 9))

data_merge[(data_merge['tariff'] == 'ultra')
           & (data_merge['mins'] !=0)]['mins'].plot(kind='hist', bins=50, alpha = 0.5)

plt.axvline(x=500, color='r')
plt.axvline(x=3000, color='g')
plt.legend(['минуты включённые в "Smart"', 'минуты включённые в "Ultra"', "Smart", "Ultra"], loc=1)

Наблюдаем нормальное распределение. Пользователи тарифа "Smart" часто выходят за пределы включенных в тариф минут. А пользователи тарифа "Ultra" даже не приближаются к пороговому значению.

In [ ]:
data_merge[(data_merge['tariff'] == 'smart')
           & (data_merge['messages_count'] !=0)]['messages_count'].plot(
    kind='hist', bins=20, title='Сообщений в месяц', figsize=(16, 9))

data_merge[(data_merge['tariff'] == 'ultra')
           & (data_merge['messages_count'] !=0)]['messages_count'].plot(
    kind='hist', bins=20, alpha = 0.5)

plt.axvline(x=50, color='r')
plt.axvline(x=1000, color='g')
plt.legend(['сообщения включенные в "Smart"', 'сообщения включенные в "Ultra"', "Smart", "Ultra"], loc=1)

 И в сообщениях пользователи тарифа "Smart" часто выходят за включенный в тариф лимит, а пользователи тарифа "Ultra" так же как и со звонками далеки от лимита.

In [ ]:
data_merge[(data_merge['tariff'] == 'smart')
           &(data_merge['mb_used'] !=0)]['mb_used'].plot(kind='hist', bins=90, title='Мегабайт в месяц', figsize=(16, 9))

data_merge[(data_merge['tariff'] == 'ultra')
           &(data_merge['mb_used'] !=0)]['mb_used'].plot(kind='hist', bins=90, alpha = 0.5)

plt.axvline(x=15360, color='r')
plt.axvline(x=30720, color='g')
plt.legend(['трафик включенный в "Smart"', 'трафик включенный в "Ultra"', "Smart", "Ultra"], loc=1)

На графике распределение Гаусса. Интернетом абоненты пользуются гораздо активнее чем прочими услугами связи. В среднем пользователи тарифа "Smart" используют 16229 мб. данных в месяц, при лимите 15360 мб.

### Вывод: 

Пользователи тарифа "Smart" используют минуты смс и траффик сверхтарифа, а пользователи тарифа "Ultra" даже не доходят до порога лимита

## Шаг 5. Проверим гипотезы

### Cредняя выручка от пользователей тарифов "Ultra" и "Smart" различается

* H₀: Средняя выручка от пользователей тарифов "Ultra" и "Smart" не различается
* H₁: Средняя выручка от пользователей тарифов "Ultra" и "Smart" различается

In [ ]:
df_smart = data_merge[data_merge['tariff'] == 'smart']
df_ultra = data_merge[data_merge['tariff'] == 'ultra']
alpha = 0.05

result = st.ttest_ind(df_smart['profit'], df_ultra['profit'])

print('p-значение:', result.pvalue)

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

In [ ]:
df_smart['profit'].mean()

In [ ]:
df_ultra['profit'].mean()

Нулевая гипотеза не подтвердилась, а значит с большой вероятностью мы можем утверждать, что пользователи тарифов приносят разное количество денег компании.

### Cредняя выручка от пользователей из Москвы отличается от выручки от пользователей из других регионов

* H₀: Средняя выручка от пользователей из Москвы не отличается от выручки от пользователей из других регионов
* H₁: Средняя выручка от пользователей из Москвы отличается от выручки от пользователей из других регионов

In [ ]:
df_moscow = data_merge[data_merge['city'] == 'Москва']
df_another_city = data_merge[data_merge['city'] != 'Москва']

abs(np.mean(df_moscow ['profit']) - np.mean(df_another_city['profit'])) / (np.var(df_another_city['profit']))

Сравниваем дисперсии критерием фишера, разница незначительна

In [ ]:
alpha = 0.05

result = st.ttest_ind(df_moscow ['profit'], df_another_city['profit'])

print('p-значение:', result.pvalue)

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

Нулевая гипотеза не отвергнута, большая вероятность того, что пользователи из Москвы тратят столько же сколько пользователи из других городов.

### Вывод: 

Cредняя выручка от пользователей тарифов "Ultra" и "Smart" действительно различается, а вот разницы между выручкой от Москвы и от регионов нет.

## Шаг 6. Общий вывод.

* Мы изучили наши таблицы, проверили их на наличие пустых значений, дубликатов и неправильных типов данных. Затем поменяли типы данных  так как многие были указаны неверно,и подготовили все таблицы для дальнейшего анализа.

* Посчитали сколько минут разговора, сколько сообщений и какой объем интернет-трафика требуется пользователям каждого тарифа в месяц. Посчитали среднее количество, дисперсию и стандартное отклонение.

* Проверили гипотезы. Убедились, что выручка тарифов Смарт и Ультра различается. Убедились, что выручка от клиентов из Москвы и других регионов не отличается.

* Изучив данные, мы выявили, что клиенты тарифа Ультра расходуют больше интернет-трафика, делают больше звонков и отправляют больше сообщений, нежели клиенты тарифа Смарт и практически не превышают лимит своего пакета, чего не скажешь о клиентах тарифа Смарт. Но по годовой выручке, компания получает больше выручки по тарифу Смарт, так как пользователей данного тарифа более, чем в два раза больше чем Ультра. Если считать по каждому пользователю, то в среднем пользователь тарифа Ультра приносит больше выручки в год, чем пользователь Смарта.

* Если считать по итоговой прибыли от тарифов Ультра и Смарт, то Смарт выгоден для компании аж на 700к рублей в год, но если считать ценность пользователя каждого тарифа, то пользователь тарифа Ультра приносит больше дохода, чем пользователь тарифа Смарт.